In [1]:
import numpy
import py_entropy

In [2]:
# Make an initial state
s = py_entropy.State(numpy.asarray([1.0] + [0]*3, dtype="complex128"))
s.get_state()

array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [3]:
# Check that building unitaries works
units = s.make_unitaries([0, 1, 2, 3])
[abs(numpy.linalg.det(units[i])) for i in range(units.shape[0])]

[1.0, 0.9999999999999999, 1.0000000000000002, 0.9999999999999998]

In [4]:
# Time 1e6 layers, alternating between offset and non-offset
%time s.apply_alternative_layers(1000000, periodic_boundaries=True)

CPU times: user 2min 23s, sys: 26.2 s, total: 2min 50s
Wall time: 8.57 s


In [5]:
# Check state is preserved
state = s.get_state()
sum(abs(state)**2)

0.9999999999997602

In [6]:
state

array([ 0.25913791-0.1077657j , -0.36969124+0.19077336j,
        0.68930733+0.05341094j,  0.38525158+0.34892967j])

In [7]:
# Make an initial state
s = py_entropy.State(numpy.asarray([1.0] + [0]*9, dtype="complex128"))
print(abs(s.get_state()))

for i in range(10):
    s.apply_layer((i % 2) == 1, periodic_boundaries=True)
    print(abs(s.get_state()))

[1. 0. 0. 0. 0. 0.]
[0.93254681 0.36104909 0.         0.         0.         0.        ]
[0.3160257  0.12363053 0.33922255 0.         0.         0.87736612]
[0.23113191 0.24846486 0.19306031 0.27892589 0.79372666 0.37385733]
[0.38434202 0.20020817 0.24274217 0.66369916 0.51701557 0.21324275]
[0.12609961 0.41460944 0.57597107 0.40948462 0.35284538 0.43390979]
[0.33782347 0.54652496 0.4527186  0.28948873 0.45647978 0.3000902 ]
[0.20350976 0.60942432 0.20117279 0.49828442 0.41896071 0.35057074]
[0.21213965 0.39506352 0.50576013 0.53643452 0.36885424 0.3454169 ]
[0.21033702 0.39602621 0.68797403 0.26504157 0.44830442 0.23321542]
[0.14546894 0.72860523 0.31508639 0.44772835 0.26601355 0.27833412]
